In [1]:
import os

class Video_itag_val():
    def __init__(self, range_beg, range_end, chunk_len, stream_tuple, chunk_type) -> None:
        self.range_beg = range_beg
        self.range_end = range_end
        self.chunk_len = chunk_len
        self.stream_tuple = stream_tuple
        self.chunk_type = chunk_type
        
def get_filelist(path):
    Filelist = []
    dirs = os.listdir(path)
    for dir in dirs:
        Filelist.append(os.path.join(path, dir))
    return Filelist, dirs

def finger_extract(path):
    # key为itag，value为list，list中每一个元素为[range_beg,range_end,len,5tuple,video/audio]
    video_itag_dict = {}
    file_paths, file_names = get_filelist(path)
    # 从每个mitm文件中提取指纹元信息
    for i in range(0, len(file_paths)):
        mitm_file = open(file_paths[i], mode='r', encoding='utf-8')
        mitm_file_data = mitm_file.read()
        info_chunks = mitm_file_data.split('------------------------\n')[:-1]
        for chunk in info_chunks:
            lines = chunk.split('\n')
            request_head = lines[1]
            # 提取itag 和 range
            itag_index_beg = int(request_head.find("itag="))
            itag_index_end = int(request_head.find("&", itag_index_beg))
            range_index_beg = int(request_head.find("range="))
            range_index_end = int(request_head.find("&", range_index_beg))
            if itag_index_beg == -1 or range_index_beg == -1:
                # print("itag or range not found")
                return
            itag = request_head[itag_index_beg + 5:itag_index_end]
            video_range = request_head[range_index_beg + 6:range_index_end]
            video_range_beg = int(video_range.split("-")[0])
            video_range_end = int(video_range.split("-")[1])
            video_itag_val = Video_itag_val(video_range_beg, video_range_end, int(video_range_end - video_range_beg + 1), file_names[i], "video")

            if itag not in video_itag_dict:
                video_itag_dict[itag] = [video_itag_val]
            else:
                video_itag_dict[itag].append(video_itag_val)
    return video_itag_dict

def chunk_sort(video_itag_dict):
    for itag in video_itag_dict:
        tuple_dist = {}
        tmp_list = []
        # 统计各五元组出现的次数，用于后续去重
        for k in range(0, len(video_itag_dict[itag])):
            if video_itag_dict[itag][k].stream_tuple not in tuple_dist:
                tuple_dist[video_itag_dict[itag][k].stream_tuple] = 1
            else:
                tuple_dist[video_itag_dict[itag][k].stream_tuple] += 1
        # 排序
        for i in range(0, len(video_itag_dict[itag])):
            for j in range(i, len(video_itag_dict[itag])):
                if video_itag_dict[itag][i].range_beg > video_itag_dict[itag][j].range_beg:
                    video_itag_dict[itag][i], video_itag_dict[itag][j] = video_itag_dict[itag][j], \
                        video_itag_dict[itag][i]
        # 去重,根据五元组出现的次数，保留出现次数多的重复块（前提是能保证连续）
        for i in range(0, len(video_itag_dict[itag]) - 1):
            flag = 0
            j = i + 1
            while video_itag_dict[itag][i].range_beg == video_itag_dict[itag][j].range_beg:
                flag = 1
                if j + 1 >= len(video_itag_dict[itag]):
                    break
                j += 1

            if flag == 1:
                if video_itag_dict[itag][i].range_end + 1 == video_itag_dict[itag][j].range_beg:
                    video_itag_dict[itag][i], video_itag_dict[itag][i + 1] = video_itag_dict[itag][i + 1], \
                        video_itag_dict[itag][i]
                    continue
                else:
                    continue
            else:
                tmp_list.append(video_itag_dict[itag][i])
        tmp_list.append(video_itag_dict[itag][len(video_itag_dict[itag]) - 1])
        video_itag_dict[itag] = tmp_list
    return video_itag_dict

Filelist, Filename = get_filelist('E:/project/Attempt/data/record/mitm')
video_itag_dict_list = {}
for i in range(len(Filelist)):
    path, name = Filelist[i], Filename[i]
    try:
        video_itag_dict = finger_extract(path)
        video_itag_dict = chunk_sort(video_itag_dict)
        with open('E:/project/Attempt/data/record/url/' + name, 'r') as f:
            url = f.read().split(',')[0]
        video_itag_dict_list[url] = video_itag_dict
    except:
        # print('{} is wrong!'.format(path))
        pass

In [2]:
import csv

with open('E:/project/Attempt/data/fingerprint/segment.csv', 'r') as f:
    reader = csv.reader(f)
    txt = list(reader)
video_list = []
for line in txt:
    video = {}
    video['url'] = line[0]
    video['itag'] = line[1]
    video['start'] = int(line[3])
    video['end'] = int(line[4])
    video['seg'] = [int(i) for i in line[6].split('/')[1:]]
    video['segsum'] = [sum(video['seg'][:i + 1]) + video['end'] for i in range(len(video['seg']))]
    try:
        video['range_end'] = [i.range_end for i in video_itag_dict_list[video['url']][video['itag']]]
        video_list.append(video)
    except:
        # print(video['url'])
        pass

In [3]:
for i in range(len(video_list)):
    video_list[i]['index_list'] = [video_list[i]['segsum'].index(j) for j in video_list[i]['range_end'] if j in video_list[i]['segsum']]
    video_list[i]['combine'] = [video_list[i]['index_list'][j + 1] - video_list[i]['index_list'][j] for j in range(len(video_list[i]['index_list']) - 1)]